In [1]:
import requests
from bs4 import BeautifulSoup
import re
import mysql.connector

In [2]:
db = mysql.connector.connect(user = 'root', passwd = '13808472aA@#', host = '127.0.0.1', database = 'CITIES')
cursor = db.cursor()
cursor.execute('SELECT * FROM CITY_')
cities = cursor.fetchall()
cities = list(map(lambda x:x[0], cities))

In [3]:
def link_miner(city):
    response = requests.get('https://divar.ir/s/%s/auto' % city)
    soup = BeautifulSoup(response.text, "html.parser")
    links = [link.get('href') for link in soup.find_all('a')][19:-1]
    return links

In [4]:

def data_miner(sublink):
    response = requests.get(f'https://divar.ir{sublink}')
    soup = BeautifulSoup(response.text, "html.parser")
    try:
        name = soup.find_all(
            "h1", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text
        name = re.findall('(.*)،', name)[0]
        milage = int(soup.find_all(
            "span", class_="kt-group-row-item__value")[0].text.replace('٬', ''))
        try:
            model = int(soup.find_all(
                "span", class_="kt-group-row-item__value")[1].text)
        except ValueError:
            model = soup.find_all(
                "span", class_="kt-group-row-item__value")[1].text
        engine_status = soup.find_all(
            "p", class_="kt-unexpandable-row__value")[0].text
        chassis_status = soup.find_all(
            "p", class_="kt-unexpandable-row__value")[1].text
        body_status = soup.find_all(
            "p", class_="kt-unexpandable-row__value")[2].text
        insurance = soup.find_all(
            "p", class_="kt-unexpandable-row__value")[3].text
        gearbox = soup.find_all(
            "p", class_="kt-unexpandable-row__value")[4].text
        try:
            price = int(soup.find_all(
                "p", class_="kt-unexpandable-row__value")[6].text.replace('٬', '')[:-6])
        except ValueError:
            price = soup.find_all(
                "p", class_="kt-unexpandable-row__value")[6].text
        return name, milage, model, engine_status, chassis_status, body_status, insurance, gearbox, price
    except IndexError:
        return None

In [8]:
data = []
for city in cities:
    print(city.title())
    for sublink in link_miner(city):
        if data_miner(sublink) != None:
            data.append(data_miner(sublink))
    print(f'The information of {city.title()} city is successfully mined.')


Tehran
The information of Tehran city is successfully mined.
Mashhad


KeyboardInterrupt: 

In [9]:
cursor.execute('CREATE DATABASE IF NOT EXISTS INFORMATION')
cursor.execute('USE INFORMATION')
cursor.execute('CREATE TABLE IF NOT EXISTS CAR(name VARCHAR(255), milage VARCHAR(255), model VARCHAR(255), engine_status VARCHAR(255), chassis_status VARCHAR(255), body_status VARCHAR(255), insurance VARCHAR(255), gearbos VARCHAR(255), price VARCHAR(255))')
for i in set(data):
    cursor.execute(f'INSERT INTO CAR VALUES({repr(i[0])}, {repr(i[1])}, {repr(i[2])}, {repr(i[3])}, {repr(i[4])}, {repr(i[5])}, {repr(i[6])}, {repr(i[7])}, {repr(i[8])})')

    db.commit()

In [10]:
cursor.execute('select * from car')
s = cursor.fetchall()
for i in s:
    print(i)

('هن تنگ X5 مونتاژ', '263', '1399', 'سالم', 'سالم و پلمپ', 'سالم و بیu200cخط و خش', '۶ ماه', 'اتوماتیک', '770000000')
('رنو 5', '300000', '1990', 'سالم', 'سالم و پلمپ', 'تمامu200cرنگ', '۱۰ ماه', 'دندهu200cای', '30000000')
('رنو ساندرو استپu200cوی اتوماتیک', '2000', '1397', 'سالم', 'سالم و پلمپ', 'سالم و بیu200cخط و خش', '۴ ماه', 'اتوماتیک', '645000000')
('پراید 132 SE', '120000', '1397', 'سالم', 'سالم و پلمپ', 'رنگu200cشدگی در ۱ ناحیه', '۶ ماه', 'دندهu200cای', '145000000')
('سمند LX ساده', '27000', '1398', 'سالم', 'سالم و پلمپ', 'سالم و بیu200cخط و خش', '۶ ماه', 'دندهu200cای', '265000000')
('پژو پارس ساده', '400000', '1386', 'سالم', 'ضربهu200cخورده', 'رنگu200cشدگی در ۳ ناحیه', '۱ ماه', 'دندهu200cای', '124000000')
('پژو پارس ساده', '200000', '1393', 'سالم', 'سالم و پلمپ', 'سالم و بیu200cخط و خش', '۶ ماه', 'دندهu200cای', '210000000')
('پراید 131 SE', '120000', '1396', 'سالم', 'سالم و پلمپ', 'خط و خش جزیی', '۹ ماه', 'دندهu200cای', '146000000')
('سمند سورن معمولی', '185000', '1392', 'سالم'